## RA Veronese Table

This script computes the solutions of optimization problems of the type $(L\pi)_{s}$, which were introduced in `Chapter 4` in the thesis, where
$$
s\colon (\mathbb R^n)^m \to S^d(\mathbb R^n), (x_1,\ldots, x_m) \mapsto \sum_{i=1}^m x_i^{\otimes d}
$$
parametrizes a Zariski dense subset of the $m$-th secant of the Veronese variety in degree $d\in \mathbb N$. In `Chapter 4`, we studied this map for $d \in \{2, 3\}$. 

In [1]:
using JuMP, COSMO, MosekTools, DynamicPolynomials, LinearAlgebra, SparseArrays, SuiteSparse, StatsBase
using DataFrames, CSV, JLD2, FileIO
include("veronese.jl");


In [2]:
function compute_projection(vars, vars2, s, u, ε, K=d)
    mon(k) = reverse(monomials(vars2, 0:k))
    model = Model(Mosek.Optimizer)
    N = length(mon(2K))
    @variable(model, λ[1:N])
    lin_vars = Dict([monom=>value for (monom, value) in zip(mon(2K),  λ[1:N])] )
    𝔼(p) = coefficients(p, collect(keys(lin_vars))) ⋅ collect(values(lin_vars))
    linearize_moment_matrix = function(moment_matrix)
        return [lin_vars[m] for m in moment_matrix];
    end
    @constraint(model, linearize_moment_matrix(mon(K)*mon(K)') in PSDCone());
    q = ε^2 - (s(vars) - u)⋅(s(vars) - u)
    @constraint(model, 𝔼(vars2[1]^0) == 1)
    if K == d
        @constraint(model, 𝔼(q) >= 0)
    else 
        @constraint(model, 𝔼.(q.*mon(K-d)*mon(K-d)') in PSDCone())
    end
    @objective(model, Min, 𝔼((s(vars) - u)⋅(s(vars) - u)))
    #MOI.set(model, MOI.Silent(), true);
    optimize!(model)
    opt = value.(𝔼((s(vars) - u)⋅(s(vars) - u)));
    avg_x = value.(𝔼.(vars2));
    avg_z = value.(𝔼.(s(vars)));
    return avg_z, avg_x, opt, model
end

compute_projection (generic function with 2 methods)

In [3]:
const d = 3;
create_projection_instance = function(m, n, ε; random=false, K=d, write=true)
    # define a parameterized variety (here, m-th secant of n-dim 3rd veronese)
    @polyvar Z[1:n]
    mon_Zd = monomials(Z, d);
    @polyvar X[1:m, 1:n];
    @polyvar X0[1:n];
    e = [eachcol(Matrix(I, n, n))...]
    vecX = [X[i, :] for i=1:m]
    s0(x) = (Z⋅x)^d
    Js0(x, h) = (Z⋅x)^(d-1) * (h⋅Z)
    s_polys = [subs(p, Z=>ones(n)) for p in terms(s0(X0))]
    s(x) = sum([p(X0=>x[i]) for p in s_polys] for i=1:m)

    # define parameters
    if random
        x0 = [randn(n) for i=1:m]; 
    else
        x0 = e[1:m];
    end
    z0 = s(x0);
    u = z0 + ε.*ν(x0, d, Z, n) ; # starting point  
    time = @elapsed z, x, opt, model = compute_projection(vecX, vec(X), s, u, 2ε, K)
    if random 
        filepath_csv = "data/veronese-deg$(d)-random-raw-$(n)$(m).csv";
        filepath_jld = "data/veronese-deg$(d)-random-raw-$(n)$(m).jld2";
    else
        filepath_csv = "data/veronese-deg$(d)-raw-$(n)$(m).csv";
        filepath_jld = "data/veronese-deg$(d)-raw-$(n)$(m).jld2";
    end
    
    id = 1;
    if isfile(filepath_csv)
        df = CSV.read(filepath_csv, DataFrame);
        id = max(df.id...)+1;
    else
        df = DataFrame(:id=>Int[], :d=>Int[], :n=>Int[], :m=>Int[], :K=>Int[], :ε=>Float64[], :time=>Float64[], :exact=>Bool[], :sqerror=>Float64[], :optval=>Float64[], :dist_after_rounding=>Float64[], :ν=>[])
        CSV.write(filepath_csv, df);
    end
    
    exact = norm(z-z0)^2<10^(-7);
    sqerror = norm(z-z0)^2;
    dist_after_rounding = norm(z-u)^2;
    if random 
        new_row = DataFrame(:id=>[id], :d=>[d], :n=>[n], :m=>[m], :K=>[K], :ε=>[2ε], :time=>[time], :exact=>[exact], :sqerror=>[sqerror], :optval=>[opt], :dist_after_rounding=>[dist_after_rounding], :u=>[u], :z=>[z], :z0=>[z0])
    else
        new_row = DataFrame(:id=>[id], :d=>[d], :n=>[n], :m=>[m], :K=>[K], :ε=>[2ε], :time=>[time], :exact=>[exact], :sqerror=>[sqerror], :optval=>[opt], :dist_after_rounding=>[dist_after_rounding], :ν=>[u-z0])
    end
    if write
        CSV.write(filepath_csv, new_row; append=true);
    end
   
#    if random 
#        save(filepath_jld, Dict("model_$(id)" => model, "z0_$(id)" => z0, "u_$(id)" => u,  "z_$(id)" => z))
#    else
#        save(filepath_jld, Dict("model_$(id)" => model, "ν_$(id)" => ν(x0, d, Z, n), "u_$(id)" => u,  "z_$(id)" => z))
#    end
    return model, z, z0, u, opt
end

#27 (generic function with 1 method)

In [4]:
# #df = CSV.read(filepath_csv, DataFrame);
df = DataFrame(:n=>Int[], :m=>Int[], :K=>Int[], :ε=>Float64[], :avgtime=>Float64[], :avgexact=>Int[], :avgsqerror => Float64[], :nsamples=>Int64[]) 
filepath_csv = "data/veronese-deg$(d).csv";
CSV.write(filepath_csv, df);


for n = 3:3
    for m=1:1
        df_raw = DataFrame();
        nsamples = 1;
        ε = 0.02;
        K = d;
        for k = 1:nsamples
            #append!(df_raw, create_projection_instance(m, n, ε; random=false));
            create_projection_instance(m, n, ε; random=false, K=K);
        end
        if false #(m==n && n≠3) || (n == 4 && m>=3)
           continue; 
        end
        df_raw = CSV.read("data/veronese-deg$(d)-raw-$(n)$(m).csv", DataFrame);
        avgtime = mean(df_raw.time)
        avgexact = mean(df_raw.exact)
        avgsqerror = mean(df_raw.sqerror)
        nsamples_total = size(df_raw, 1);
        new_row = DataFrame(:n=>[n], :m=>[m], :K=>[K], :ε=>[ε], :avgtime=>[avgtime], :avgexact=>[avgexact], :avgsqerror =>[avgsqerror], :nsamples=>[nsamples_total])
        CSV.write(filepath_csv, new_row; append=true);
    end
end
        

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 212             
  Cones                  : 0               
  Scalar variables       : 84              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective se